In [12]:
from scapy.all import *

In [13]:
pckts = sniff(timeout = 5)

In [14]:
pckts[0].show()

###[ Ethernet ]###
  dst       = ff:ff:ff:ff:ff:ff
  src       = 0c:9a:3c:49:11:41
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 96
     id        = 21045
     flags     = 
     frag      = 0
     ttl       = 128
     proto     = udp
     chksum    = 0xbabb
     src       = 172.31.85.94
     dst       = 172.31.127.255
     \options   \
###[ UDP ]###
        sport     = netbios_ns
        dport     = netbios_ns
        len       = 76
        chksum    = 0x1fec
###[ NBNS Header ]###
           NAME_TRN_ID= 58552
           RESPONSE  = 0
           OPCODE    = 5
           NM_FLAGS  = B+RD
           RCODE     = 0
           QDCOUNT   = 1
           ANCOUNT   = 0
           NSCOUNT   = 0
           ARCOUNT   = 1
###[ NBNS registration request ]###
              QUESTION_NAME= b'MSI'
              SUFFIX    = workstation
              NULL      = 0
              QUESTION_TYPE= NB
              QUESTION_CLASS= INTERNET
          

In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [8]:
path = r"..\API\data\raw\Train_data.csv"
df = pd.read_csv(path)

In [7]:
# unzip C:\Users\dheer\Desktop\ML\Network_Intrusion\API\data\raw\archive.zip
!Expand-Archive -Path ".\API\data\raw\archive.zip" -DestinationPath ".\API\data\raw"


'Expand-Archive' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
df.sample(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
14311,5,tcp,smtp,SF,775,338,0,0,0,0,...,175,0.62,0.03,0.01,0.01,0.0,0.0,0.0,0.0,normal
9690,0,udp,domain_u,SF,42,147,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,normal
3358,0,icmp,eco_i,SF,18,0,0,0,0,0,...,255,1.00,0.00,1.00,0.25,0.0,0.0,0.0,0.0,anomaly
11659,0,tcp,http,SF,333,4023,0,0,0,0,...,255,1.00,0.00,0.00,0.02,0.0,0.0,0.0,0.0,normal
7619,14,tcp,smtp,SF,1119,330,0,0,0,0,...,172,0.59,0.04,0.01,0.02,0.0,0.0,0.0,0.0,normal


In [10]:
df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'class'],
      dtype='object')

In [17]:
from scapy.all import *

def extract_features(packet):
    features = {}

    # Basic packet details
    features['duration'] = 0  # Needs session tracking (handled externally)
    features['protocol_type'] = packet.proto if packet.haslayer(IP) else "unknown"
    features['service'] = packet.dport if packet.haslayer(TCP) or packet.haslayer(UDP) else "unknown"
    features['flag'] = packet.sprintf("%TCP.flags%") if packet.haslayer(TCP) else "unknown"

    # Packet sizes
    features['src_bytes'] = len(packet.payload)
    features['dst_bytes'] = len(packet.payload.payload)

    # TCP-specific features
    features['land'] = int(packet.haslayer(IP) and packet[IP].src == packet[IP].dst)
    features['wrong_fragment'] = packet[IP].frag if packet.haslayer(IP) else 0
    features['urgent'] = packet[TCP].urgptr if packet.haslayer(TCP) else 0

    # Additional extracted features (these require tracking across packets)
    features['hot'] = 0  # Example: number of suspicious operations (custom definition needed)
    features['num_failed_logins'] = 0  # Requires session tracking
    features['logged_in'] = 1 if packet.haslayer(TCP) and packet[TCP].flags == "S" else 0
    features['root_shell'] = 0  # Needs payload inspection
    features['su_attempted'] = 0  # Needs session tracking

    # Network-based features
    features['count'] = 1  # Count of packets in a session (needs tracking)
    features['srv_count'] = 1  # Needs service tracking

    # Error rates (Require multiple packet analysis)
    features['serror_rate'] = 0  # Syn flag errors (requires tracking)
    features['srv_serror_rate'] = 0
    features['rerror_rate'] = 0
    features['srv_rerror_rate'] = 0

    # Host-based features (require tracking of previous connections)
    features['dst_host_count'] = 1
    features['dst_host_srv_count'] = 1
    features['dst_host_same_srv_rate'] = 1
    features['dst_host_diff_srv_rate'] = 0
    features['dst_host_same_src_port_rate'] = 1
    features['dst_host_srv_diff_host_rate'] = 0

    # Label (Not available in real traffic, but in datasets)
    features['class'] = "unknown"  # You must classify it based on ML models

    return features

# Extract features from a captured packet
sniffed_packets = sniff(count=5)
for pkt in sniffed_packets:
    extracted_features = extract_features(pkt)
    print(extracted_features)


{'duration': 0, 'protocol_type': 17, 'service': 22222, 'flag': 'unknown', 'src_bytes': 56, 'dst_bytes': 36, 'land': 0, 'wrong_fragment': 0, 'urgent': 0, 'hot': 0, 'num_failed_logins': 0, 'logged_in': 0, 'root_shell': 0, 'su_attempted': 0, 'count': 1, 'srv_count': 1, 'serror_rate': 0, 'srv_serror_rate': 0, 'rerror_rate': 0, 'srv_rerror_rate': 0, 'dst_host_count': 1, 'dst_host_srv_count': 1, 'dst_host_same_srv_rate': 1, 'dst_host_diff_srv_rate': 0, 'dst_host_same_src_port_rate': 1, 'dst_host_srv_diff_host_rate': 0, 'class': 'unknown'}
{'duration': 0, 'protocol_type': 17, 'service': 22222, 'flag': 'unknown', 'src_bytes': 56, 'dst_bytes': 36, 'land': 0, 'wrong_fragment': 0, 'urgent': 0, 'hot': 0, 'num_failed_logins': 0, 'logged_in': 0, 'root_shell': 0, 'su_attempted': 0, 'count': 1, 'srv_count': 1, 'serror_rate': 0, 'srv_serror_rate': 0, 'rerror_rate': 0, 'srv_rerror_rate': 0, 'dst_host_count': 1, 'dst_host_srv_count': 1, 'dst_host_same_srv_rate': 1, 'dst_host_diff_srv_rate': 0, 'dst_host_